# Líneas base de las sedes

In [1]:
# device = 'K0303'
# active_power_variable = 'active_power'
path_base_r_string = r'D:\OneDrive - CELSIA S.A E.S.P'

%load_ext autoreload
%autoreload 2
import warnings
# warnings.filterwarnings("ignore")

In [2]:
# this cell enables project path relative imports
import sys
from pathlib import Path
path_base = Path(path_base_r_string)
project_path = path_base / 'Proyectos' / 'Eficiencia_Energetica' / 'Homecenter' / 'Informe_01'
sys.path.append(str(project_path))

In [3]:
# import all your modules here
import json
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import config as cfg

from library_ubidots_v2 import Ubidots as ubi
from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro


In [4]:
df = pd.read_pickle(project_path / "dataframe.pkl")
df_tag_aa_ilu = pd.read_excel(project_path / "ubidots_device_map.xlsx", sheet_name='AA-ILU')

# for some reason 'value' dtypes aren't consistent...
df['value'] = pd.to_numeric(df['value'], errors='coerce')

# merge both tables
df = (
    pd.merge(
        df.reset_index(),
        df_tag_aa_ilu,
        how='inner'
    )
    .set_index('datetime')
) 

In [5]:
dates_to_remove = {
    
    'cali-norte---torre-de-enfriamiento':[
        ['2022-01-01','2022-05-25']
    ],
    'cali-sur---aires-acondicionados':[
        ['2022-01-01','2022-05-25']
    ],
    'cedritos---iluminacion-patio-constructor':[
        ['2022-01-01','2022-05-20']
    ],
    
    'cali-norte---iluminacion-parqueaderos-2':[
        ['2022-01-01','2022-04-30']
    ],
    'cali-norte---iluminacion-principal-2':[
        ['2022-01-01','2022-05-25']
    ],
    'iluminacion---primer-piso':[ # cedritos
        ['2022-01-01','2022-12-30']
    ],
    'cedritos-iluminacion-principal':[
        ['2022-05-15','2022-12-30']
    ],
    'bello---iluminacion-patio-constructor':[
        ['2022-01-01','2022-05-25']
    ],
    'tintal---iluminacion-parqueaderos':[
        ['2022-01-01','2022-06-20']
    ],
    'baq---aire-acondicionado-piso-ventas':[
        ['2022-01-01','2022-05-20']
    ],
}

energy_var_rename = {
    'ener':'energia-activa-acumulada',
    'energia-activa':'energia-activa',
    'energia-activa-acumulada':'energia-activa-acumulada',
    'energia-activa-calculada':'energia-activa'
}

In [6]:
# get (unified) cummulative energy from consumption
df_cons = df.query('variable == "energia-activa"').copy()
df_ea = cln.recover_energy_from_consumption(df_cons, new_varname='energia-activa-acumulada')

# get power from energy
df_pa_synth = cln.differentiate_single_variable(df_ea, 'potencia-activa-sintetica', remove_gap_data=True)

# clean power data
df_pa_synth = cln.remove_outliers_by_zscore(df_pa_synth, zscore=5)
df_pa_synth = cln.subset_discard_date_intervals(df_pa_synth, dates_to_remove)
df_pa_synth = pro.datetime_attributes(df_pa_synth)
df_pa_synth = df_pa_synth.query("value >= 0")

In [7]:
df_pa_synth

,value,variable,device,device_name,type,office_label,hour,day,dow,cont_dow,week,month,year
datetime,,,,,,,,,,,,,
2022-04-01 01:00:00-05:00,1.68,potencia-activa-sintetica,bello-mini-splits,Bello - Mini Splits,AA,hc-bello,1,1,viernes,4.041667,13,4,2022
2022-04-01 02:00:00-05:00,1.65,potencia-activa-sintetica,bello-mini-splits,Bello - Mini Splits,AA,hc-bello,2,1,viernes,4.083333,13,4,2022
2022-04-01 03:00:00-05:00,1.69,potencia-activa-sintetica,bello-mini-splits,Bello - Mini Splits,AA,hc-bello,3,1,viernes,4.125000,13,4,2022
2022-04-01 04:00:00-05:00,1.69,potencia-activa-sintetica,bello-mini-splits,Bello - Mini Splits,AA,hc-bello,4,1,viernes,4.166667,13,4,2022
2022-04-01 05:00:00-05:00,1.69,potencia-activa-sintetica,bello-mini-splits,Bello - Mini Splits,AA,hc-bello,5,1,viernes,4.208333,13,4,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-27 01:00:00-05:00,2.64,potencia-activa-sintetica,cedritos---iluminacion-sotanos,Cedritos - Iluminación Sótanos,ILU,cedritos,1,27,jueves,3.041667,43,10,2022
2022-10-27 02:00:00-05:00,3.56,potencia-activa-sintetica,cedritos---iluminacion-sotanos,Cedritos - Iluminación Sótanos,ILU,cedritos,2,27,jueves,3.083333,43,10,2022
2022-10-27 05:00:00-05:00,3.51,potencia-activa-sintetica,cedritos---iluminacion-sotanos,Cedritos - Iluminación Sótanos,ILU,cedritos,5,27,jueves,3.208333,43,10,2022


## Resultados:

<!-- Plotly curva horaria por circuito en la sede:
- Para AA
- Para Iluminación
- Para el total

Curva de KPI (kWh/m^2/año) por sede

Series de tiempo de consumos mensuales (bar plot)

Figura de consumo energético por tipo de día de la semana



Bell y San Juan dejan la iluminación prendida -->


In [ ]:
def office_cell_function(df_office, office_name):

    df_day_by_hour = (
        df_office
        .reset_index()
        .groupby(['office_label','type','device_name','hour'])['value']
        .agg(['median','mean','std','min',pro.q_low,pro.q_high,'max','count'])
        .reset_index()
    )

    df_week_by_day = (
        df_office
        .reset_index()
        .groupby(['office_label','type','device_name','cont_dow'])['value']
        .agg(['median','mean','std','min',pro.q_low,pro.q_high,'max','count'])
        .reset_index()
    )

    grp.plot_typical_day_by_hour(
        df_day_by_hour.query("type == 'AA'"), 
        subset='device_name', 
        title=f"Día típico para los equipos de climatización en la sede de {office_name}", 
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )

    grp.plot_typical_day_by_hour(
        df_day_by_hour.query("type == 'ILU'"), 
        subset='device_name', 
        title=f"Día típico para la iluminación en la sede de {office_name}", 
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )
    
    grp.plot_typical_week_by_day(
        df_week_by_day.query("type == 'AA'"),
        subset='device_name',  
        title=f"Semana típica para los equipos de climatización en la sede de {office_name}",
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )

    grp.plot_typical_week_by_day(
        df_week_by_day.query("type == 'ILU'"),
        subset='device_name',  
        title=f"Semana típica para la iluminación en la sede de {office_name}",
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )

### Barranquilla

In [ ]:
office_name = 'Barranquilla'
office_label = 'hc-baq'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Bello

In [ ]:
office_name = 'Bello'
office_label = 'hc-bello'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Bucaramanga

In [ ]:
office_name = 'Bucaramanga'
office_label = 'hc---bucaramanga'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Cali Norte

In [ ]:
office_name = 'Cali Norte'
office_label = 'hc---cali-norte'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Cali Sur

In [ ]:
office_name = 'Cali Sur'
office_label = 'hc-cali-sur'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Calle 80

In [ ]:
office_name = 'Calle 80'
office_label = 'calle-80'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Cedritos

In [ ]:
office_name = 'Cedritos'
office_label = 'cedritos'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Funza

In [ ]:
office_name = 'Funza'
office_label = 'hc-funza'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### La Popa

In [ ]:
office_name = 'La Popa'
office_label = 'hc-la-popa'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Palmira

In [ ]:
office_name = 'Palmira'
office_label = 'hc---palmira'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### San Fernando

In [ ]:
office_name = 'San Fernando'
office_label = 'hc-san-fernando'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### San Juan

In [ ]:
office_name = 'San Juan'
office_label = 'hc-san-juan'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)

### Tintal

In [ ]:
office_name = 'Tintal'
office_label = 'hc-tintal'
df_office = df_pa_synth.query("office_label == @office_label")
office_cell_function(df_office, office_name)